## Lesson 2: Connecting with a CRM

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


## Start of lesson

In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [4]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: I'm sorry, but I'm unable to process your request for a refund on the
       broken mug at this time. I will need to transfer you to a human
       customer support agent who can properly investigate the issue
       and initiate the refund process if appropriate. Please hold
       while I transfer you.

Session ID: 1b317861-ff01-4711-abe6-56375b2127bb


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; The file that is examined in the video is at  <code>./ro_shared_data/functions/lambda_stage_1.py</code> </p>

In [5]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [6]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [7]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': '79d52d57-58d4-425c-9a21-d8d813ec09c5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 11 Aug 2025 14:42:10 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '79d52d57-58d4-425c-9a21-d8d813ec09c5',
   'x-amz-apigw-id': 'PJUicHOFvHcEPXA=',
   'x-amzn-trace-id': 'Root=1-689a0142-675f5bfd6d417ef155da3830'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:637423403721:function:dlai-support-agent-M2WAR'},
  'actionGroupId': 'R7N419FK1G',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': 'WXQS5XYSOX',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 8, 11, 14, 42, 10, 503013, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is privat

In [8]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [9]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

In [10]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [12]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [13]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: I'm sorry, but I don't have enough information to fully process your
       request for a refund on the broken mug. Could you please
       provide some additional details, such as:  - When and where did
       you purchase the mug? - Do you have a receipt or order number?
       - Can you describe the issue with the mug in more detail?  With
       this additional information, I can better assist you and
       escalate your request to our support team. Please let me know
       if you can provide any of those details.

Session ID: 524bf6d6-03ca-4738-9aa3-40fca0688433


In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  I already have the customer's name and email address, but I still
  need more details to properly escalate this request for a refund on
  the broken mug.

Observation:
  Type: FINISH

Final response:
  Thank you for providing your name and email address, Mike. However,
  I still need some additional details to process your refund request:
  - When and where did you purchase the mug? - Do you have a receipt
  or order number for the purchase? - Can you describe the issue with
  the mug in more detail, such as how it became broken?  With this
  extra information, I can better understand the situation and
  escalate your request to our support team. Please let me know if you
  can provide any of those details.


Session ID: 524bf6d6-03ca-4738-9aa3-40fca0688433
